In [ ]:
import airsim
import numpy as np
import cv2
import imutils
import matplotlib.pyplot as plt

# connect to the AirSim simulator
client = airsim.MultirotorClient()
client.confirmConnection()
client.enableApiControl(True)
client.armDisarm(True)

In [ ]:
def move(img_rgb):
    image=img_rgb
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY_INV)[1]
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    for c in cnts:
    # compute the center of the contour
        M = cv2.moments(c)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        l = [cX,cY]
        return l
    

client.takeoffAsync().join()
client.moveToPositionAsync(-10,-10, -10, 5).join()
#client.moveToPositionAsync(-5, 10, -10, 5).join()

responses = client.simGetImages([airsim.ImageRequest("0",airsim.ImageType.DepthVis,False,False)])
response = responses[0]
img1d = np.frombuffer(response.image_data_uint8, dtype=np.uint8)
img_rgb = img1d.reshape(response.height, response.width, 3)
while(true):
    # ret, frame = cap.read()
    c=move(img_rgb)
    cx,cy = c[0],c[1]
    client.moveToPositionAsync(cx, cy, -10, 5).join()
    break
    
